## The data

A large technology firm have been hacked! Luckily their forensic engineers have grabbed valuable data about the hacks, including information like session time,locations, wpm typing speed, etc. The forensic engineer relates to you what she has been able to figure out so far, she has been able to grab meta data of each session that the hackers used to connect to their servers. These are the features of the data:

* 'Session_Connection_Time': How long the session lasted in minutes


* 'Bytes Transferred': Number of MB transferred during session


* 'Kali_Trace_Used': Indicates if the hacker was using Kali Linux


* 'Servers_Corrupted': Number of server corrupted during the attack


* 'Pages_Corrupted': Number of pages illegally accessed


* 'Location': Location attack came from (Probably useless because the hackers used VPNs)


* 'WPM_Typing_Speed': Their estimated typing speed based on session logs.

The technology firm has 3 potential hackers that perpetrated the attack. Their certain of the first two hackers but they aren't very sure if the third hacker was involved or not.

One last key fact, the forensic engineer knows that the hackers trade off attacks. Meaning they should each have roughly the same amount of attacks. For example if there were 100 total attacks, then in a 2 hacker situation each should have about 50 hacks, in a three hacker situation each would have about 33 hacks.

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('hack').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/21 20:37:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
data = spark.read.csv("hack_data.csv", header=True, inferSchema=True)

In [3]:
data.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)



In [4]:
data.head(1)

[Row(Session_Connection_Time=8.0, Bytes Transferred=391.09, Kali_Trace_Used=1, Servers_Corrupted=2.96, Pages_Corrupted=7.0, Location='Slovenia', WPM_Typing_Speed=72.37)]

In [5]:
data.describe().show()

24/01/21 20:37:30 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+
|summary|Session_Connection_Time| Bytes Transferred|   Kali_Trace_Used|Servers_Corrupted|   Pages_Corrupted|   Location|  WPM_Typing_Speed|
+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+
|  count|                    334|               334|               334|              334|               334|        334|               334|
|   mean|     30.008982035928145| 607.2452694610777|0.5119760479041916|5.258502994011977|10.838323353293413|       NULL|57.342395209580864|
| stddev|     14.088200614636158|286.33593163576757|0.5006065264451406| 2.30190693339697|  3.06352633036022|       NULL| 13.41106336843464|
|    min|                    1.0|              10.0|                 0|              1.0|               6.0|Afghanistan|              40.0|
|    max|           

## Data transformation and scaling

In [6]:
from pyspark.ml.feature import VectorAssembler, StandardScaler

/Users/schanlar/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [7]:
assembler = VectorAssembler(
    inputCols=[
        "Session_Connection_Time",
        "Bytes Transferred",
        "Kali_Trace_Used",
        "Servers_Corrupted",
        "Pages_Corrupted",
        "WPM_Typing_Speed",
    ],
    outputCol="features",
)

In [8]:
output = assembler.transform(data)

In [9]:
output.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)
 |-- features: vector (nullable = true)



In [10]:
final_data = output.select('features')

In [11]:
final_data.printSchema()

root
 |-- features: vector (nullable = true)



In [12]:
scaler = StandardScaler(inputCol='features', outputCol='scaledFeatures')

In [13]:
scaled_data = scaler.fit(final_data).transform(final_data)

In [14]:
scaled_data.show(5)

+--------------------+--------------------+
|            features|      scaledFeatures|
+--------------------+--------------------+
|[8.0,391.09,1.0,2...|[0.56785108466505...|
|[20.0,720.99,0.0,...|[1.41962771166263...|
|[31.0,356.32,1.0,...|[2.20042295307707...|
|[2.0,228.08,1.0,2...|[0.14196277116626...|
|[20.0,408.5,0.0,3...|[1.41962771166263...|
+--------------------+--------------------+
only showing top 5 rows



## Create a Clustering Model

In [15]:
from pyspark.ml.clustering import KMeans

In [16]:
kmeans2 = KMeans(featuresCol='scaledFeatures', k=2)
kmeans3 = KMeans(featuresCol='scaledFeatures', k=3)

In [17]:
k2_model = kmeans2.fit(scaled_data)
k3_model = kmeans3.fit(scaled_data)

24/01/21 20:37:34 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/01/21 20:37:34 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS


## Evaluating K2 v K3 Models

In [18]:
from pyspark.ml.evaluation import ClusteringEvaluator

In [19]:
# wssse
print(f"Training cost for KMeans with 2 clusters: {k2_model.summary.trainingCost:.2f}")
print(f"Training cost for KMeans with 3 clusters: {k3_model.summary.trainingCost:.2f}")

Training cost for KMeans with 2 clusters: 601.77
Training cost for KMeans with 3 clusters: 434.76


In [20]:
k2_model_preds = k2_model.transform(scaled_data)
k3_model_preds = k3_model.transform(scaled_data)

In [21]:
k2_model_preds.show(5)

+--------------------+--------------------+----------+
|            features|      scaledFeatures|prediction|
+--------------------+--------------------+----------+
|[8.0,391.09,1.0,2...|[0.56785108466505...|         0|
|[20.0,720.99,0.0,...|[1.41962771166263...|         0|
|[31.0,356.32,1.0,...|[2.20042295307707...|         0|
|[2.0,228.08,1.0,2...|[0.14196277116626...|         0|
|[20.0,408.5,0.0,3...|[1.41962771166263...|         0|
+--------------------+--------------------+----------+
only showing top 5 rows



In [22]:
k3_model_preds.show(5)

+--------------------+--------------------+----------+
|            features|      scaledFeatures|prediction|
+--------------------+--------------------+----------+
|[8.0,391.09,1.0,2...|[0.56785108466505...|         1|
|[20.0,720.99,0.0,...|[1.41962771166263...|         1|
|[31.0,356.32,1.0,...|[2.20042295307707...|         1|
|[2.0,228.08,1.0,2...|[0.14196277116626...|         1|
|[20.0,408.5,0.0,3...|[1.41962771166263...|         1|
+--------------------+--------------------+----------+
only showing top 5 rows



In [23]:
evaluator = ClusteringEvaluator()

In [24]:
k2_model_silhouette = evaluator.evaluate(k2_model_preds)
k3_model_silhouette = evaluator.evaluate(k3_model_preds)

print(f"K2 model silhouette with squared euclidean distance : {k2_model_silhouette:.3f}")
print(f"K3 model silhouette with squared euclidean distance : {k3_model_silhouette:.3f}")

K2 model silhouette with squared euclidean distance : 0.668
K3 model silhouette with squared euclidean distance : 0.304


Not much to be gained from the WSSSE, after all, we would expect that as K increases, the WSSSE decreases.

However, we know that the attacks should be evenly numbered between the hackers! Let's check with the transform and prediction columns that result form this!

In [25]:
k2_model_preds.groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  167|
|         0|  167|
+----------+-----+



In [26]:
k3_model_preds.groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  167|
|         2|   79|
|         0|   88|
+----------+-----+



**Conclusion**: It was 2 hackers, in fact, our clustering algorithm created two equally sized clusters with K=2